In [6]:

import jsonlines
import ndjson
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pecos

!{sys.executable} -m pip install numpy


Matplotlib is building the font cache; this may take a moment.


# 1 Import the libriaries

Using Anaconda or other python manager to install the above libraries. On anaconda use cmd: conda install ....python package ..

It is most efficient using pip install for the jsonlines and ndjson packages

# 2 Set the array variables for OpenAQ dataset 

In [7]:
data = []

# 3 Import the OpenAQ Dataset

### 3 It uses the OpenAQ Dataset of a day in 2018 

### 3 The default use OpenAQ_1.ndjson (a newline delimited format)

### 3 Choose another one of the OpenAQ datasets in ndjson by changing the address of open('......')

In [10]:
with jsonlines.open('OpenAQ_1.ndjson') as reader:
    for obj in reader:
        #print(obj)
        obj1 = pd.Series(obj)
        #pd.read_csv(obj)
        data.append(obj1)


# 4 Import OpenAQ dataset to Pandas DataFrame

In [11]:
Dataset = pd.DataFrame(data)

# 5 Define the ploting method

In [12]:
def plot_df(df, x, y, title="", xlabel='Date', ylabel='Value', dpi=100):
    plt.figure(figsize=(16,5), dpi=dpi)
    plt.plot(x, y, color='tab:red')
    plt.gca().set(title=title, xlabel=xlabel, ylabel=ylabel)
    plt.show()

# 6 Choose and print some variables from Pandas Dataframe

In [13]:
print(Dataset[['date','value','parameter','location']])

print(Dataset.dtypes)

                                                    date   value parameter  \
0      {'utc': '2018-01-01T00:00:00.000Z', 'local': '...   135.7       so2   
1      {'utc': '2018-01-01T00:00:00.000Z', 'local': '...    37.3       no2   
2      {'utc': '2018-01-01T00:00:00.000Z', 'local': '...     3.4        o3   
3      {'utc': '2018-01-01T00:00:00.000Z', 'local': '...   163.5      pm25   
4      {'utc': '2018-01-01T00:00:00.000Z', 'local': '...   200.3       so2   
...                                                  ...     ...       ...   
16229  {'utc': '2018-01-01T00:00:00.000Z', 'local': '...    15.0       so2   
16230  {'utc': '2018-01-01T00:00:00.000Z', 'local': '...    55.0       no2   
16231  {'utc': '2018-01-01T00:00:00.000Z', 'local': '...   691.0        co   
16232  {'utc': '2017-05-29T20:00:00.000Z', 'local': '...  1000.0      pm10   
16233  {'utc': '2017-05-29T20:00:00.000Z', 'local': '...     4.0       so2   

         location  
0           Skver  
1           Skver  
2  

# 7 Split the 'date' variable to two variables because it has utc and local dates in one variable

In [14]:
#How to split column into two columns
#https://www.geeksforgeeks.org/split-a-text-column-into-two-columns-in-pandas-dataframe/

Dataset[['Dateutc','Datelocal']] = Dataset.date.apply(lambda x: pd.Series(str(x).split(",")))

Dataset_split = Dataset.Dateutc.apply(lambda x: pd.Series(str(x).split(":")))

#print(Dataset_split)


# 8 Convert new 'utc' (Date in utc) variable to date format for Pandas

In [15]:





Dataset['utc'] = pd.to_datetime(Dataset_split[1])

#print(Dataset.dtypes)

#print(Dataset)

pd.to_datetime(Dataset['utc'])

0       2018-01-01 00:00:00
1       2018-01-01 00:00:00
2       2018-01-01 00:00:00
3       2018-01-01 00:00:00
4       2018-01-01 00:00:00
                ...        
16229   2018-01-01 00:00:00
16230   2018-01-01 00:00:00
16231   2018-01-01 00:00:00
16232   2017-05-29 20:00:00
16233   2017-05-29 20:00:00
Name: utc, Length: 16234, dtype: datetime64[ns]

# 9 Copy OpenAQ dataset 

In [21]:
Dataset2 = Dataset[['utc','value','parameter','location']].copy()

pd.to_datetime(Dataset2['utc'])
#Dataset2.set_index('utc')

0       2018-01-01 00:00:00
1       2018-01-01 00:00:00
2       2018-01-01 00:00:00
3       2018-01-01 00:00:00
4       2018-01-01 00:00:00
                ...        
16229   2018-01-01 00:00:00
16230   2018-01-01 00:00:00
16231   2018-01-01 00:00:00
16232   2017-05-29 20:00:00
16233   2017-05-29 20:00:00
Name: utc, Length: 16234, dtype: datetime64[ns]

# 10 Copy OpenAQ Dataset variable 'location' 

In [22]:
Location_Subset = Dataset[['location']].copy()

# 11 Sort descending and remove duplicate to get all unique station's 'location'

In [23]:
Location_Subset.sort_values('location', ascending=False)

Location_Subset = Location_Subset.drop_duplicates(subset='location', keep='first')

# 12 Set the OpenAQ dataset variable 'utc' (The date in utc) to the index

In [24]:
Dataset2.index = Dataset2['utc']

# 13 The setting the array variables results 

In [25]:
Test_results_location_subset = []

i = 0

pm2 = []


# 14 The iteration over every station 'location' in the Location_Subset array 

# 15 Iterate over every AQ variable i.e. No2, So2, PM10, PM25 that the location has 

## 15 It only iterates over the AQ variables that are measured at the choosen location

In [26]:

for LocationId in Location_Subset['location']:
 
    print(LocationId)   
  
    Test_results_location_subset_parameter = []
    
    # 14 i Append the location being searched to array variable 
    
    Test_results_location_subset_parameter.append(LocationId)
    
    # 14 ii Filter the OpenAQ Dataset for the location being searched for   
    
    Dataset3 = Dataset2[Dataset2['location']==LocationId]
    
    # 14 iii Format the utc variable to Date format of Pandas
    
    pd.to_datetime(Dataset3['utc'])
    
    #14 iv Copy all the AQ variables of searched for station
    
    Test_results_location_subset_parameter_subset = Dataset3[['parameter']].copy() 
         
    #print(Test_results_location_subset_parameter_subset)
    
    #14 v Sort and remove duplicates getting unique AQ Variables
    
    Test_results_location_subset_parameter_subset.sort_values('parameter', ascending=False)
    
    Test_results_location_subset_parameter_subset = Test_results_location_subset_parameter_subset.drop_duplicates(subset='parameter', keep='first')

    #15 Iterate over the AQ Variables 
    
    for parameter in Test_results_location_subset_parameter_subset['parameter']:
        
        print(parameter)
        
        #15 i Append AQ variable to array
        
        Test_results_parameter = []
        
        Test_results_parameter.append(parameter)
        
        # 15 ii Filter the OpenAQ dataset for AQ Variable 
        
        Dataset4 = Dataset3[Dataset3['parameter']==parameter]      

        # 15 iv The get variable of Date from utc variable
        
        Dataset4['year'] = Dataset4['utc'].dt.year
        
        Dataset4['month'] = Dataset4['utc'].dt.month

        Dataset4['day'] = Dataset4['utc'].dt.day
        
        pd.to_datetime(Dataset4['utc'])

        Dataset5 = Dataset4;

        #15 v The Pecos package to find errors in the OpenAQ dataset choosen on criteria
        
        # 15 v Initialize logger
        pecos.logger.initialize()

        # 15 v Create a Pecos PerformanceMonitoring data object
        pm1 = pecos.monitoring.PerformanceMonitoring()
        
        pm1.add_dataframe(Dataset4)
        
        # 15 vi Check for missing values
    
        pm1.check_missing()
        
        # 15 vii Check for corrupt data values
        
        ###### Change value -999.00000 to other value that are not accepted 
        
        pm1.check_corrupt([-999.000000],'value') 

# Add a composite signal which compares measurements to a model
#wave_model = np.array(np.sin(10*clock_time/86400))
#wave_measurments = pm.df[pm.trans['Wave']]
#wave_error = np.abs(wave_measurments.subtract(wave_model,axis=0))
#wave_error.columns=['Wave Error C', 'Wave Error D']
#pm.add_dataframe(wave_error)
#pm.add_translation_dictionary({'Wave Error': ['Wave Error C', 'Wave Error D']})

        # 15 viii Check data for expected ranges
        
        #### CHOOSE criteria ### Change [0,10000000] to selected lower and high bound ###
        
        pm1.check_range([0, 10000000], 'value')

        # 15 iix Check that the Date utc value is accurate i.e. expected measurements

        pm1.check_range([2017, 2018], 'year')
        pm1.check_corrupt([2,3,4,5,6,7,8,9,10,11],'month')
        pm1.check_corrupt([2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30],'day')
        
        # 15 ix Append results to Array

        print(pm1.test_results)
        
        Test_results_parameter.append(len(pm1.test_results))
        
        Test_results_parameter.append(pm1.test_results)
        Test_results_parameter.append(Dataset5)
        
        Test_results_location_subset_parameter.append(Test_results_parameter)
        
        
        
        #i = i + 1
       

    Test_results_location_subset.append(Test_results_location_subset_parameter)
    


Skver
so2


C:\Users\Gordon\Miniconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gordon\Miniconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Gordon\Miniconda3\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

Empty DataFrame
Columns: [Variable Name, Start Time, End Time, Timesteps, Error Flag]
Index: []
no2


Check for corrupt data
Check for missing data
Check for corrupt data
Check for data outside expected range
Check for data outside expected range
Check for corrupt data


Empty DataFrame
Columns: [Variable Name, Start Time, End Time, Timesteps, Error Flag]
Index: []
o3


...

Empty DataFrame
Columns: [Variable Name, Start Time, End Time, Timesteps, Error Flag]
Index: []
Yatağan
pm10


Check for corrupt data
Check for missing data
Check for corrupt data
Check for data outside expected range
Check for data outside expected range
Check for corrupt data


  Variable Name          Start Time            End Time Timesteps  \
0         month 2017-05-29 20:00:00 2017-05-29 20:00:00         2   
1           day 2017-05-29 20:00:00 2017-05-29 20:00:00         2   

     Error Flag  
0  Corrupt data  
1  Corrupt data  
so2


Check for corrupt data


  Variable Name          Start Time            End Time Timesteps  \
0         month 2017-05-29 20:00:00 2017-05-29 20:00:00         2   
1           day 2017-05-29 20:00:00 2017-05-29 20:00:00         2   

     Error Flag  
0  Corrupt data  
1  Corrupt data  


# 16 Print Results

In [ ]:
print(Test_results_location_subset)

# 17 Output results to Excel

In [ ]:
for Test_results_location in Test_results_location_subset:
    
    for Test_Parameter in Test_results_location[1:]:
        
        pecos.io.write_test_results(Test_Parameter)